In [4]:
import numpy as np
from tqdm import trange
from astropy.io import ascii
from astropy.table import Table
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import warnings
from astroquery.gaia import Gaia

chrome_options = Options()
import astropy.units as u
from astropy.coordinates import SkyCoord
from astroquery.gaia import Gaia

Gaia.MAIN_GAIA_TABLE = "gaiadr2.gaia_source"
#chrome_options.add_argument("--disable-extensions")
#chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")  # linux only
chrome_options.add_argument("--headless")

In [5]:
def search_gaia(gaia_id, RA=None, dec=None, star=None):
    coord = SkyCoord(ra=RA, dec=dec, unit=(u.hourangle, u.deg), frame='icrs')
    width = u.Quantity(0.1, u.deg)
    height = u.Quantity(0.1, u.deg)
    r = Gaia.query_object_async(coordinate=coord, width=width, height=height)
    i = np.where(r['DESIGNATION'] == gaia_id)
    Gmag = float(r['phot_g_mean_mag'][i])
    BPmag = float(r['phot_bp_mean_mag'][i])
    RPmag = float(r['phot_rp_mean_mag'][i])
    pmRA_gaia = f"{float(r['pmra'][i]): .4f}"
    pmdec_gaia = f"{float(r['pmdec'][i]): .4f}"

    BP_RP = BPmag - RPmag
    Rmag = Gmag + 0.29676 - 0.64728 * (BP_RP) + 0.10141 * (BP_RP) ** 2
    Rmag = f'{Rmag: .4f}'
    if BP_RP > 0.2 and BP_RP < 4.5:
        return Rmag, pmRA_gaia, pmdec_gaia
    else:
        print('The r mag is: ' + str(i))
        warnings.warn(f'This Bp-Rp range (BP-RP = {BP_RP}) may not be valid. ')
        return Rmag, pmRA_gaia, pmdec_gaia


def search_exofop(star):
    driver = webdriver.Chrome()  # '/home/tehan/Downloads/chromedriver-linux64/chromedriver'
    driver.get('https://exofop.ipac.caltech.edu/tess/')
    search = driver.find_element(By.NAME, value='target')
    search.send_keys(star)
    search.send_keys(Keys.ENTER)
    time.sleep(2)

    # RA/dec
    position = driver.find_elements(By.TAG_NAME, 'li')
    gaia_id = None
    for j in range(len(position)):
        if position[j].text.split(' ')[0] == 'RA/Dec':
            RA = position[j].text.split(' ')[4]
            dec = position[j].text.split(' ')[5]
        elif position[j].text.split(' ')[0] == 'Star':
            for k in range(len(position[j].text.split(', '))):
                if position[j].text.split(', ')[k][:4] == 'Gaia':
                    gaia_id = position[j].text.split(', ')[k]

    # proper motion
    table = driver.find_element(By.CLASS_NAME, 'inline-table')
    tableRows = table.find_element(By.TAG_NAME, 'tr')
    pmRA = tableRows.text.split(' ')[1]
    pmdec = tableRows.text.split(' ')[4]

    # magnitudes
    Jmag = '0'
    Vmag = '0'
    Rmag = '0'
    Tmag = '0'
    magnitudes = driver.find_element(By.ID, 'myGrid6')
    mags = np.array(magnitudes.text.split("\n"))
    try:
        Jmag = mags[np.where(mags == 'J')[0][0] + 1].split(' ')[0]
    except:
        pass
    try:
        Vmag = mags[np.where(mags == 'V')[0][0] + 1].split(' ')[0]
    except:
        pass
    try:
        Rmag = mags[np.where(mags == 'r')[0][0] + 1].split(' ')[0]
    except IndexError:
        try:
            Rmag, pmRA_gaia, pmdec_gaia = search_gaia(gaia_id, RA=RA, dec=dec)
        except ValueError:
            print(f'No Gaia ID for {star}.')
            pass
    try:
        Tmag = mags[np.where(mags == 'TESS')[0][0] + 1].split(' ')[0]
    except:
        pass
    driver.quit()

    # if pmRA == '0':
    #     pmRA = pmRA_gaia
    # if pmdec == '0':
    #     pmdec = pmdec_gaia

    return [star, RA, dec, pmRA, pmdec, Jmag, Vmag, Rmag, Tmag]

# element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "myDynamicElement")))

In [6]:
path = '/home/tehan/Documents/ShaneAO/'
# data = ascii.read(f'{path}shaneAO_proposal.csv', data_start=0)
stars = [305506996, 404456775]

object_list = []
for i in trange(len(stars)):
    object_list.append(search_exofop(stars[i]))

# change the wrong minus sign
# for lists in object_list:
#     for sub in lists:
#         sub.replace('−', '-')

object_list = Table(np.array(object_list), names=['Name', 'RA', 'Dec', 'pmra', 'pmdec', 'Jmag', 'Vmag', 'Rmag', 'Tmag'])
ascii.write(object_list, f'{path}object_list.csv', format='csv', overwrite=True)
object_list

100%|██████████| 2/2 [00:15<00:00,  7.81s/it]


Name,RA,Dec,pmra,pmdec,Jmag,Vmag,Rmag,Tmag
str21,str21,str21,str21,str21,str21,str21,str21,str21
305506996,21:41:11.88,+09:35:56.59,-8.93319,-25.5916,13.118,16.602,16.0885,14.5249
404456775,22:15:34.66,+16:17:07.41,62.8829,-35.2633,13.009,16.29,16.2776,14.5372
